# CNN from scratch

This notebook aims to solve Lung CT segmentation using Convolutional Neural Network.

First, some imports.

In [1]:
import numpy as np
from utils.data_utils import read_data
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from keras.layers import Cropping2D, Concatenate, Activation
from keras.optimizers import RMSprop
from keras.models import Model

Using TensorFlow backend.


Then, read the data.

In [3]:
x_train, y_train = read_data()

# check data size
x_train = x_train.reshape((266,512,512,1))
y_train = y_train.reshape((266,512,512,1))
print(x_train.shape, y_train.shape)

(266, 512, 512, 1) (266, 512, 512, 1)


# U-Net

First, we use the popular U-Net structure.

In [ ]:
# build the model
# down path
inputs = Input(shape=(512,512,1))
x = Conv2D(64, (3, 3), padding='same', activation='relu')(inputs) #(512,512,64)
map1 = Conv2D(64, (3, 3), padding='same', activation='relu')(x) #(512,512,64)

x = MaxPooling2D()(map1) #(256,256,64)
x = Conv2D(128, (3, 3), padding='same', activation='relu')(x) #(256,256,128)
map2 = Conv2D(128, (3, 3), padding='same', activation='relu')(x) #(256,256,128)

x = MaxPooling2D()(map2) #(128,128,128)
x = Conv2D(256, (3, 3), padding='same', activation='relu')(x) #(128,128,256)
map3 = Conv2D(256, (3, 3), padding='same', activation='relu')(x) #(128,128,256)

x = MaxPooling2D()(map3) #(64,64,256)
x = Conv2D(512, (3, 3), padding='same', activation='relu')(x) #(64,64,512)
map4 = Conv2D(512, (3, 3), padding='same', activation='relu')(x) #(64,64,512)

x = MaxPooling2D()(map4) #(32,32,512)
x = Conv2D(1024, (3, 3), padding='same', activation='relu')(x) #(32,32,1024)
x = Conv2D(1024, (3, 3), padding='same', activation='relu')(x) #(32,32,1024)

# up path
x = UpSampling2D()(x) #(64,64,1024)
x = Conv2D(512, (2, 2), padding='same', activation='relu')(x) #(64,64,512)
x = Concatenate(axis=-1)([map4, x]) #(64,64,1024)
x = Conv2D(512, (3, 3), padding='same', activation='relu')(x) #(64,64,512)
x = Conv2D(512, (3, 3), padding='same', activation='relu')(x) #(64,64,512)

x = UpSampling2D()(x) #(128,128,512)
x = Conv2D(256, (2, 2), padding='same', activation='relu')(x) #(128,128,256)
x = Concatenate(axis=-1)([map3, x]) #(128,128,512)
x = Conv2D(256, (3, 3), padding='same', activation='relu')(x) #(128,128,256)
x = Conv2D(256, (3, 3), padding='same', activation='relu')(x) #(128,128,256)

x = UpSampling2D()(x) #(256,256,256)
x = Conv2D(128, (2, 2), padding='same', activation='relu')(x) #(256,256,128)
x = Concatenate(axis=-1)([map2, x]) #(256,256,256)
x = Conv2D(128, (3, 3), padding='same', activation='relu')(x) #(256,256,128)
x = Conv2D(128, (3, 3), padding='same', activation='relu')(x) #(256,256,128)

x = UpSampling2D()(x) #(512,512,128)
x = Conv2D(64, (2, 2), padding='same', activation='relu')(x) #(512,512,64)
x = Concatenate(axis=-1)([map1, x]) #(512,512,128)
x = Conv2D(64, (3, 3), padding='same', activation='relu')(x) #(512,512,64)
x = Conv2D(64, (3, 3), padding='same', activation='relu')(x) #(512,512,64)
predictions = Conv2D(1, (1,1), activation='softmax')(x) #(512,512,1)

unet = Model(inputs=inputs, outputs=predictions)

# set loss and optimizer
rmsprop = RMSprop(lr=0.001, decay=0.99)
unet.compile(optimizer=rmsprop,
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
unet.fit(x_train, y_train,
         batch_size=16,
         initial_epoch=0,
         epochs=10,
         callbacks=[])